# IMPORT DATA

In [1]:
# import libraries
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt

# import files
fname1 = 'C:/Users/Felix/Documents/GitHub/naturalDisasterAnalysis/Data/emdat_public_2021_06_24_query_uid-s5YWMa.csv' # EMDAT natural disaster dataset
EMDAT = pd.read_csv(fname1, dtype = 'string')

fname2 = 'C:/Users/Felix/Documents/GitHub/naturalDisasterAnalysis/Data/CPIAUCNS.csv' # FRED consumer price index dataset
inflation = pd.read_csv(fname2)

In [2]:
# configure plot settings
#pd.set_option('display.max_rows', None)

#SMALL_SIZE = 10
#MEDIUM_SIZE = 15
#LARGE_SIZE = 20

#plt.rc('font', size=SMALL_SIZE)          # controls default text sizes
#plt.rc('axes', titlesize=LARGE_SIZE)     # fontsize of the axes title
#plt.rc('axes', labelsize=LARGE_SIZE)    # fontsize of the x and y labels
#plt.rc('xtick', labelsize=MEDIUM_SIZE)    # fontsize of the tick labels
#plt.rc('ytick', labelsize=MEDIUM_SIZE)    # fontsize of the tick labels
#plt.rc('legend', fontsize=MEDIUM_SIZE)    # legend fontsize
#plt.rc('figure', titlesize=LARGE_SIZE)  # fontsize of the figure title

# CLEAN DATA

In [3]:
# filter out irrelevant disasters
EMDAT = EMDAT[EMDAT['Disaster Group'] != 'Technological']
EMDAT = EMDAT[EMDAT['Disaster Subgroup'] != 'Extra-terrestrial']
EMDAT = EMDAT[EMDAT['Disaster Subgroup'] != 'Biological']
EMDAT = EMDAT[EMDAT['Disaster Subgroup'] != 'Complex Disasters']
EMDAT = EMDAT[EMDAT['Disaster Type'] != 'Epidemic']
EMDAT = EMDAT[EMDAT['Disaster Type'] != 'Mass movement (dry)']

# turn string values into integers
EMDAT['Year'] = [int(year) for year in EMDAT['Year']]
EMDAT['Total Damages'] = EMDAT['Total Damages'].fillna('0')
EMDAT['Total Damages'] = [int(cost) for cost in EMDAT['Total Damages']]
EMDAT['Reconstruction Costs'] = EMDAT['Reconstruction Costs'].fillna('0')
EMDAT['Reconstruction Costs'] = [int(cost) for cost in EMDAT['Reconstruction Costs']]
EMDAT['Insured Damages'] = EMDAT['Insured Damages'].fillna('0')
EMDAT['Insured Damages'] = [int(cost) for cost in EMDAT['Insured Damages']]

# take data from last 40 years as recommended by Centre de Recherche sur L’Epidemiologie des Désastres (CRED)
EMDAT = EMDAT[EMDAT['Year'] >= 1980]
EMDAT = EMDAT[EMDAT['Year'] < 2021]

# filter out irrelevant columns
columns = ['Year', 'Country', 'Disaster Group', 'Disaster Subgroup', 'Disaster Type', 'Disaster Subtype', 'Disaster Subsubtype', 'Total Damages']
data = EMDAT[columns]
# count number of disasters in each year
disaster_counts = data['Year'].value_counts().sort_index()

# get mean CPI and CPI multiplier by year
inflation = inflation.groupby('Year').mean()

# merge tables
data = data.merge(inflation, on = 'Year')

# normalize prices to 2020 prices
data['Total Damages'] = data['Total Damages'] * data['CPI Multiplier']

# set prices from K USD to B USD
data['Total Damages'] = data['Total Damages'] / 1000000

# create new dataframes for each subgroup
hydrological_data = data[data['Disaster Subgroup'] == 'Hydrological']

# count number of disasters in each year for each subgroup
hydrological_disaster_counts = hydrological_data['Year'].value_counts().sort_index()

# save index of years
hydrological_disaster_counts_index = hydrological_disaster_counts.index

# sum costs by year
hydrological_costs = hydrological_data[['Year', 'Total Damages']]
hydrological_costs = hydrological_costs.groupby('Year').sum()
hydrological_costs = hydrological_costs.squeeze()

# SPLIT HYDROLOGICAL DISASTERS

In [4]:
# display unique types
for dtype in hydrological_data['Disaster Type'].unique():
    print(dtype)

Flood
Landslide


In [5]:
# create new dataframes for each type
flood_data = hydrological_data[hydrological_data['Disaster Type'] == 'Flood']
landslide_data = hydrological_data[hydrological_data['Disaster Type'] == 'Landslide']

# count number of disasters in each year for each subgroup
flood_disaster_counts = flood_data['Year'].value_counts().sort_index()
landslide_disaster_counts = landslide_data['Year'].value_counts().sort_index()

# save index of years
flood_disaster_counts_index = flood_disaster_counts.index
landslide_disaster_counts_index = landslide_disaster_counts.index

# HYDROLOGICAL DISASTER OCCURRENCE ANALYSIS

In [ ]:
# single line plot

plt.grid(axis = 'both')
plt.title("Hydrological Disasters per Year\n")
plt.xlabel("\nYear")
plt.ylabel("Number of Hydrological Disasters\n")
plt.plot(hydrological_disaster_counts_index,hydrological_disaster_counts, linewidth=2, color = 'lightseagreen', alpha = 0.9, label = 'Total hydrological')
plt.plot(flood_disaster_counts_index, flood_disaster_counts, linestyle = '--', color = 'darkseagreen', alpha = 0.9, label = 'Flood')
plt.plot(landslide_disaster_counts_index, landslide_disaster_counts, linestyle = '--', color = 'sienna', alpha = 0.9, label = 'Landslide')

lgd = plt.legend(bbox_to_anchor=(1,1), loc = 'upper left')

plt.savefig("C:/Users/Felix/Documents/GitHub/naturalDisasterAnalysis/Hydrological Disasters/hydrological figures/hydrologicalDisastersPerYearSLP.png", bbox_extra_artists=(lgd,), bbox_inches='tight')

In [ ]:
# multiple line plots

fig, axs = plt.subplots(3, sharex = True)
fig.suptitle("Hydrological Disasters per Year\n")
axs[1].set_ylabel("Number of Hydrological Disasters\n")
axs[2].set_xlabel("\nYear")

axs[0].plot(hydrological_disaster_counts_index, hydrological_disaster_counts, linewidth=2, color = 'lightseagreen', alpha = 0.9, label = 'Total Hydrological')
axs[1].plot(flood_disaster_counts_index, flood_disaster_counts, linestyle = '--', color = 'darkseagreen', alpha = 0.9, label = 'Flood')
axs[2].plot(landslide_disaster_counts_index, landslide_disaster_counts, linestyle = '--', color = 'sienna', alpha = 0.9, label = 'Landslide')

plt.sca(axs[0])
plt.yticks(np.linspace(0,200,3))
plt.grid(axis = 'both')

plt.sca(axs[1])
plt.yticks(np.linspace(0,200,3))
plt.grid(axis = 'both')

plt.sca(axs[2])
plt.yticks(np.linspace(0,30,3))
plt.grid(axis = 'both')

lgd = fig.legend(bbox_to_anchor=(1,1), loc = 'upper left')
plt.savefig("C:/Users/Felix/Documents/GitHub/naturalDisasterAnalysis/Hydrological Disasters/hydrological figures/hydrologicalDisastersPerYearMLP.png", bbox_extra_artists=(lgd,), bbox_inches='tight')

In [ ]:
# single bar chart

plt.grid(axis = 'y')
plt.title("Hydrological Disasters per Year\n")
plt.xlabel("\nYear")
plt.ylabel("Number of Hydrological Disasters\n")

plt.bar(flood_disaster_counts_index, flood_disaster_counts, edgecolor = 'black', color = 'darkseagreen', alpha = 0.9, label = 'Flood')
plt.bar(landslide_disaster_counts_index, landslide_disaster_counts, bottom = flood_disaster_counts, edgecolor = 'black', color = 'sienna', alpha = 0.9, label = 'Landslide')

lgd = plt.legend(bbox_to_anchor=(1,1), loc = 'upper left')

plt.savefig("C:/Users/Felix/Documents/GitHub/naturalDisasterAnalysis/Hydrological Disasters/hydrological figures/hydrologicalDisastersPerYearSBC.png", bbox_extra_artists=(lgd,), bbox_inches='tight')

In [ ]:
# multiple bar charts

fig, axs = plt.subplots(3, sharex = True)
fig.suptitle("Hydrological Disasters per Year\n")
axs[1].set_ylabel("Number of Hydrological Disasters\n")
axs[2].set_xlabel("\nYear")

axs[0].bar(hydrological_disaster_counts_index, hydrological_disaster_counts, edgecolor = 'black', color = 'lightseagreen', alpha = 0.9, label = 'Total Hydrological')
axs[1].bar(flood_disaster_counts_index, flood_disaster_counts, edgecolor = 'black', color = 'darkseagreen', alpha = 0.9, label = 'Flood')
axs[2].bar(landslide_disaster_counts_index, landslide_disaster_counts, edgecolor = 'black', color = 'sienna', alpha = 0.9, label = 'Landslide')

plt.sca(axs[0])
plt.yticks(np.linspace(0,200,3))
plt.grid(axis = 'y')

plt.sca(axs[1])
plt.yticks(np.linspace(0,200,3))
plt.grid(axis = 'y')

plt.sca(axs[2])
plt.yticks(np.linspace(0,30,3))
plt.grid(axis = 'y')

lgd = fig.legend(bbox_to_anchor=(1,1), loc = 'upper left')

plt.savefig("C:/Users/Felix/Documents/GitHub/naturalDisasterAnalysis/hydrological Disasters/hydrological figures/hydrologicalDisastersPerYearMBC.png", bbox_extra_artists=(lgd,), bbox_inches='tight')

In [ ]:
# BOXPLOT HYDROLOGICAL DISASTER OCCURRENCES

total_box_data = [hydrological_disaster_counts[0:10], hydrological_disaster_counts[10:20], hydrological_disaster_counts[20:30], hydrological_disaster_counts[30:40]]
box_labels = ['1980s', '1990s', '2000s', '2010s']

x0 = [np.random.normal(1,0.04,10)]
x1 = [np.random.normal(2,0.04,10)]
x2 = [np.random.normal(3,0.04,10)]
x3 = [np.random.normal(4,0.04,10)]

plt.figure(figsize=(3,6))
plt.title("Hydrological Disasters per Year")
plt.xlabel("Decade")
plt.ylabel("Number of Hydrological Disasters")

plt.boxplot(total_box_data, labels = box_labels, whis = [0, 100])

plt.scatter(x0,total_box_data[0], color = 'lightseagreen', edgecolor = 'black', alpha = 0.9)
plt.scatter(x1,total_box_data[1], color = 'lightseagreen', edgecolor = 'black', alpha = 0.9)
plt.scatter(x2,total_box_data[2], color = 'lightseagreen', edgecolor = 'black', alpha = 0.9)
plt.scatter(x3,total_box_data[3], color = 'lightseagreen', edgecolor = 'black', alpha = 0.9)

plt.savefig("C:/Users/Felix/Documents/GitHub/naturalDisasterAnalysis/Hydrological Disasters/hydrological figures/hydrologicalDisastersPerYearBOX.png", bbox_inches='tight')

In [ ]:
# BOXPLOT FLOOD OCCURRENCES

total_box_data = [flood_disaster_counts[0:10], flood_disaster_counts[10:20], flood_disaster_counts[20:30], flood_disaster_counts[30:40]]
box_labels = ['1980s', '1990s', '2000s', '2010s']

x0 = [np.random.normal(1,0.04,10)]
x1 = [np.random.normal(2,0.04,10)]
x2 = [np.random.normal(3,0.04,10)]
x3 = [np.random.normal(4,0.04,10)]

plt.figure(figsize=(3,6))
plt.title("Flood Disasters per Year")
plt.xlabel("Decade")
plt.ylabel("Number of Flood Disasters")

plt.boxplot(total_box_data, labels = box_labels, whis = [0, 100])

plt.scatter(x0,total_box_data[0], color = 'darkseagreen', edgecolor = 'black', alpha = 0.9)
plt.scatter(x1,total_box_data[1], color = 'darkseagreen', edgecolor = 'black', alpha = 0.9)
plt.scatter(x2,total_box_data[2], color = 'darkseagreen', edgecolor = 'black', alpha = 0.9)
plt.scatter(x3,total_box_data[3], color = 'darkseagreen', edgecolor = 'black', alpha = 0.9)

plt.savefig("C:/Users/Felix/Documents/GitHub/naturalDisasterAnalysis/Hydrological Disasters/hydrological figures/floodsPerYearBOX.png", bbox_inches='tight')

In [ ]:
# BOXPLOT LANDSLIDE OCCURRENCES

total_box_data = [landslide_disaster_counts[0:10], landslide_disaster_counts[10:20], landslide_disaster_counts[20:30], landslide_disaster_counts[30:40]]
box_labels = ['1980s', '1990s', '2000s', '2010s']

x0 = [np.random.normal(1,0.04,10)]
x1 = [np.random.normal(2,0.04,10)]
x2 = [np.random.normal(3,0.04,10)]
x3 = [np.random.normal(4,0.04,10)]

plt.figure(figsize=(3,6))
plt.title("Landslide Disasters per Year")
plt.xlabel("Decade")
plt.ylabel("Number of Landslide Disasters")

plt.boxplot(total_box_data, labels = box_labels, whis = [0, 100])

plt.scatter(x0,total_box_data[0], color = 'sienna', edgecolor = 'black', alpha = 0.9)
plt.scatter(x1,total_box_data[1], color = 'sienna', edgecolor = 'black', alpha = 0.9)
plt.scatter(x2,total_box_data[2], color = 'sienna', edgecolor = 'black', alpha = 0.9)
plt.scatter(x3,total_box_data[3], color = 'sienna', edgecolor = 'black', alpha = 0.9)

plt.savefig("C:/Users/Felix/Documents/GitHub/naturalDisasterAnalysis/Hydrological Disasters/hydrological figures/landslidesPerYearBOX.png", bbox_inches='tight')

# HYDROLOGICAL DISASTER OCCURRENCE STATISTICS

In [ ]:
# HYDROLOGICAL DISASTER OCCURRENCE STATISTICS

print("HYDROLOGICAL DISASTER OCCURRENCE STATISTICS")

# hydrological statistical data
print("\nTotal Hydrological Statistical Data: ")
print(hydrological_disaster_counts.describe())

# 1980s hydrological statistical data
print("\n1980s Hydrological Statistical Data: ")
print(hydrological_disaster_counts[0:10].describe())

# 1990s hydrological statistical data
print("\n1990s Hydrological Statistical Data: ")
print(hydrological_disaster_counts[10:20].describe())

# 2000s hydrological statistical data
print("\n2000s Hydrological Statistical Data: ")
print(hydrological_disaster_counts[20:30].describe())

# 2010s hydrological statistical data
print("\n2010s Hydrological Statistical Data: ")
print(hydrological_disaster_counts[30:40].describe())

print("\n")

# hydrological statistical changes 1980s-2010s
hydrological_mean_1980s = hydrological_disaster_counts[0:10].mean()
hydrological_mean_2010s = hydrological_disaster_counts[30:40].mean()
mean_percent_change_hydrological_1980_to_2020 = 100*(hydrological_mean_2010s-hydrological_mean_1980s)/hydrological_mean_1980s
print("Mean Hydrological Disaster Change (1980s-2010s): {:.2f}%".format(mean_percent_change_hydrological_1980_to_2020))

hydrological_max_1980s = hydrological_disaster_counts[0:10].max()
hydrological_max_2010s = hydrological_disaster_counts[30:40].max()
max_percent_change_hydrological_1980_to_2020 = 100*(hydrological_max_2010s-hydrological_max_1980s)/hydrological_max_1980s
print("Max Hydrological Disaster Change (1980s-2010s): {:.2f}%".format(max_percent_change_hydrological_1980_to_2020))

hydrological_min_1980s = hydrological_disaster_counts[0:10].min()
hydrological_min_2010s = hydrological_disaster_counts[30:40].min()
min_percent_change_hydrological_1980_to_2020 = 100*(hydrological_min_2010s-hydrological_min_1980s)/hydrological_min_1980s
print("Min Hydrological Disaster Change (1980s-2010s): {:.2f}%".format(min_percent_change_hydrological_1980_to_2020))

hydrological_std_1980s = hydrological_disaster_counts[0:10].std()
hydrological_std_2010s = hydrological_disaster_counts[30:40].std()
std_percent_change_hydrological_1980_to_2020 = 100*(hydrological_std_2010s-hydrological_std_1980s)/hydrological_std_1980s
print("Standard Deviation of Hydrological Disasters Change (1980s-2010s): {:.2f}%\n".format(std_percent_change_hydrological_1980_to_2020))

In [ ]:
# FLOOD DISASTER OCCURRENCE STATISTICS

print("FLOOD DISASTER OCCURRENCE STATISTICS")

# flood statistical data
print("\nTotal Flood Statistical Data: ")
print(flood_disaster_counts.describe())

# 1980s flood statistical data
print("\n1980s Flood Statistical Data: ")
print(flood_disaster_counts[0:10].describe())

# 1990s flood statistical data
print("\n1990s Flood Statistical Data: ")
print(flood_disaster_counts[10:20].describe())

# 2000s flood statistical data
print("\n2000s Flood Statistical Data: ")
print(flood_disaster_counts[20:30].describe())

# 2010s flood statistical data
print("\n2010s Flood Statistical Data: ")
print(flood_disaster_counts[30:40].describe())

print("\n")

# flood statistical changes 1980s-2010s
flood_mean_1980s = flood_disaster_counts[0:10].mean()
flood_mean_2010s = flood_disaster_counts[30:40].mean()
mean_percent_change_flood_1980_to_2020 = 100*(flood_mean_2010s-flood_mean_1980s)/flood_mean_1980s
print("Mean Flood Disaster Change (1980s-2010s): {:.2f}%".format(mean_percent_change_flood_1980_to_2020))

flood_max_1980s = flood_disaster_counts[0:10].max()
flood_max_2010s = flood_disaster_counts[30:40].max()
max_percent_change_flood_1980_to_2020 = 100*(flood_max_2010s-flood_max_1980s)/flood_max_1980s
print("Max Flood Disaster Change (1980s-2010s): {:.2f}%".format(max_percent_change_flood_1980_to_2020))

flood_min_1980s = flood_disaster_counts[0:10].min()
flood_min_2010s = flood_disaster_counts[30:40].min()
min_percent_change_flood_1980_to_2020 = 100*(flood_min_2010s-flood_min_1980s)/flood_min_1980s
print("Min Flood Disaster Change (1980s-2010s): {:.2f}%".format(min_percent_change_flood_1980_to_2020))

flood_std_1980s = flood_disaster_counts[0:10].std()
flood_std_2010s = flood_disaster_counts[30:40].std()
std_percent_change_flood_1980_to_2020 = 100*(flood_std_2010s-flood_std_1980s)/flood_std_1980s
print("Standard Deviation of Flood Disasters Change (1980s-2010s): {:.2f}%\n".format(std_percent_change_flood_1980_to_2020))

In [ ]:
# LANDSLIDE DISASTER OCCURRENCE STATISTICS

print("LANDSLIDE DISASTER ANALYSIS")

# landslide statistical data
print("\nTotal Landslide Statistical Data: ")
print(landslide_disaster_counts.describe())

# 1980s landslide statistical data
print("\n1980s Landslide Statistical Data: ")
print(landslide_disaster_counts[0:10].describe())

# 1990s landslide statistical data
print("\n1990s Landslide Statistical Data: ")
print(landslide_disaster_counts[10:20].describe())

# 2000s landslide statistical data
print("\n2000s Landslide Statistical Data: ")
print(landslide_disaster_counts[20:30].describe())

# 2010s landslide statistical data
print("\n2010s Landslide Statistical Data: ")
print(landslide_disaster_counts[30:40].describe())

print("\n")

# landslide statistical changes 1980s-2010s
landslide_mean_1980s = landslide_disaster_counts[0:10].mean()
landslide_mean_2010s = landslide_disaster_counts[30:40].mean()
mean_percent_change_landslide_1980_to_2020 = 100*(landslide_mean_2010s-landslide_mean_1980s)/landslide_mean_1980s
print("Mean Landslide Disaster Change (1980s-2010s): {:.2f}%".format(mean_percent_change_landslide_1980_to_2020))

landslide_max_1980s = landslide_disaster_counts[0:10].max()
landslide_max_2010s = landslide_disaster_counts[30:40].max()
max_percent_change_landslide_1980_to_2020 = 100*(landslide_max_2010s-landslide_max_1980s)/landslide_max_1980s
print("Max Landslide Disaster Change (1980s-2010s): {:.2f}%".format(max_percent_change_landslide_1980_to_2020))

landslide_min_1980s = landslide_disaster_counts[0:10].min()
landslide_min_2010s = landslide_disaster_counts[30:40].min()
min_percent_change_landslide_1980_to_2020 = 100*(landslide_min_2010s-landslide_min_1980s)/landslide_min_1980s
print("Min Landslide Disaster Change (1980s-2010s): {:.2f}%".format(min_percent_change_landslide_1980_to_2020))

landslide_std_1980s = landslide_disaster_counts[0:10].std()
landslide_std_2010s = landslide_disaster_counts[30:40].std()
std_percent_change_landslide_1980_to_2020 = 100*(landslide_std_2010s-landslide_std_1980s)/landslide_std_1980s
print("Standard Deviation of Landslide Disasters Change (1980s-2010s): {:.2f}%\n".format(std_percent_change_landslide_1980_to_2020))

# YoY CHANGE IN HYDROLOGICAL DISASTER OCCURRENCES ANALYSIS

In [ ]:
# CALCULATE YoY CHANGE IN HYDROLOGICAL DISASTER OCCURRENCES

hydrological_disaster_change = [hydrological_disaster_counts[i+1]-hydrological_disaster_counts[i] for i in range(1980,2020)]
hydrological_disaster_change = pd.Series(hydrological_disaster_change)

flood_disaster_change = [flood_disaster_counts[i+1]-flood_disaster_counts[i] for i in range(1980,2020)]
flood_disaster_change = pd.Series(flood_disaster_change)

landslide_disaster_change = [landslide_disaster_counts[i+1]-landslide_disaster_counts[i] for i in range(1980,2020)]
landslide_disaster_change = pd.Series(landslide_disaster_change)

In [ ]:
# single bar chart

change_data = np.array([flood_disaster_change, landslide_disaster_change])

data_shape = np.shape(change_data)

# Take negative and positive data apart and cumulate
def get_cumulated_array(change_data, **kwargs):
    cum = change_data.clip(**kwargs)
    cum = np.cumsum(cum, axis=0)
    d = np.zeros(np.shape(change_data))
    d[1:] = cum[:-1]
    return d  

cumulated_data = get_cumulated_array(change_data, min=0)
cumulated_data_neg = get_cumulated_array(change_data, max=0)

# Re-merge negative and positive data.
row_mask = (change_data<0)
cumulated_data[row_mask] = cumulated_data_neg[row_mask]
data_stack = cumulated_data

cols = ["darkseagreen", "sienna"]
labs = ["Flood", "Landslide"]

fig = plt.figure()
ax = plt.subplot(111)

for i in np.arange(0, data_shape[0]):
    ax.bar(range(1981, 2021), change_data[i], bottom=data_stack[i], edgecolor = 'black', color=cols[i], alpha = 0.9, label = labs[i])

plt.grid(axis = 'y')

plt.title("Year over Year Change in Hydrological Disasters\n")
plt.xlabel("\nYear")
plt.ylabel("Year over Year Change\n")

lgd = plt.legend(bbox_to_anchor=(1,1), loc = 'upper left')

plt.savefig("C:/Users/Felix/Documents/GitHub/naturalDisasterAnalysis/Hydrological Disasters/hydrological figures/hydrologicalDisasterChangePerYearSBC.png", bbox_extra_artists=(lgd,), bbox_inches='tight')

In [ ]:
# YoY CHANGE IN HYDROLOGICAL DISASTER OCCURRENCES BOXPLOT

total_box_data = [hydrological_disaster_change[0:10], hydrological_disaster_change[10:20], hydrological_disaster_change[20:30], hydrological_disaster_change[30:40]]
box_labels = ['1980s', '1990s', '2000s', '2010s']

x0 = [np.random.normal(1,0.04,10)]
x1 = [np.random.normal(2,0.04,10)]
x2 = [np.random.normal(3,0.04,10)]
x3 = [np.random.normal(4,0.04,10)]

plt.figure(figsize=(3,6))
plt.title("Year over Year Change in Number of Hydrological Disasters\n")
plt.xlabel("\nDecade")
plt.ylabel("Year over Year Change\n")

plt.boxplot(total_box_data, labels = box_labels, whis = [0, 100])

plt.scatter(x0,total_box_data[0], color = 'lightseagreen', edgecolor = 'black', alpha = 0.9)
plt.scatter(x1,total_box_data[1], color = 'lightseagreen', edgecolor = 'black', alpha = 0.9)
plt.scatter(x2,total_box_data[2], color = 'lightseagreen', edgecolor = 'black', alpha = 0.9)
plt.scatter(x3,total_box_data[3], color = 'lightseagreen', edgecolor = 'black', alpha = 0.9)

plt.savefig("C:/Users/Felix/Documents/GitHub/naturalDisasterAnalysis/Hydrological Disasters/hydrological figures/hydrologicalDisasterChangePerYearBOX.png", bbox_inches='tight')

In [ ]:
# YoY CHANGE IN FLOOD OCCURRENCES BOXPLOT

total_box_data = [flood_disaster_change[0:10], flood_disaster_change[10:20], flood_disaster_change[20:30], flood_disaster_change[30:40]]
box_labels = ['1980s', '1990s', '2000s', '2010s']

x0 = [np.random.normal(1,0.04,10)]
x1 = [np.random.normal(2,0.04,10)]
x2 = [np.random.normal(3,0.04,10)]
x3 = [np.random.normal(4,0.04,10)]

plt.figure(figsize=(3,6))
plt.title("Year over Year Change in Number of Floods\n")
plt.xlabel("\nDecade")
plt.ylabel("Year over Year Change\n")

plt.boxplot(total_box_data, labels = box_labels, whis = [0, 100])

plt.scatter(x0,total_box_data[0], color = 'darkseagreen', edgecolor = 'black', alpha = 0.9)
plt.scatter(x1,total_box_data[1], color = 'darkseagreen', edgecolor = 'black', alpha = 0.9)
plt.scatter(x2,total_box_data[2], color = 'darkseagreen', edgecolor = 'black', alpha = 0.9)
plt.scatter(x3,total_box_data[3], color = 'darkseagreen', edgecolor = 'black', alpha = 0.9)

plt.savefig("C:/Users/Felix/Documents/GitHub/naturalDisasterAnalysis/Hydrological Disasters/hydrological figures/floodChangePerYearBOX.png", bbox_inches='tight')

In [ ]:
# YoY CHANGE IN LANDSLIDE OCCURRENCES BOXPLOT

total_box_data = [landslide_disaster_change[0:10], landslide_disaster_change[10:20], landslide_disaster_change[20:30], landslide_disaster_change[30:40]]
box_labels = ['1980s', '1990s', '2000s', '2010s']

x0 = [np.random.normal(1,0.04,10)]
x1 = [np.random.normal(2,0.04,10)]
x2 = [np.random.normal(3,0.04,10)]
x3 = [np.random.normal(4,0.04,10)]

plt.figure(figsize=(3,6))
plt.title("Year over Year Change in Number of Landslide Disasters\n")
plt.xlabel("\nDecade")
plt.ylabel("Year over Year Change\n")

plt.boxplot(total_box_data, labels = box_labels, whis = [0, 100])

plt.scatter(x0,total_box_data[0], color = 'sienna', edgecolor = 'black', alpha = 0.9)
plt.scatter(x1,total_box_data[1], color = 'sienna', edgecolor = 'black', alpha = 0.9)
plt.scatter(x2,total_box_data[2], color = 'sienna', edgecolor = 'black', alpha = 0.9)
plt.scatter(x3,total_box_data[3], color = 'sienna', edgecolor = 'black', alpha = 0.9)

plt.savefig("C:/Users/Felix/Documents/GitHub/naturalDisasterAnalysis/Hydrological Disasters/hydrological figures/landslideChangePerYearBOX.png", bbox_inches='tight')

# YoY CHANGE IN HYDROLOGICAL DISASTER OCCURRENCES STATISTICS

In [ ]:
# CHANGE IN HYDROLOGICAL DISASTER OCCURRENCES ANALYSIS

print("CHANGE IN HYDROLOGICAL DISASTER OCCURRENCES ANALYSIS")

# total statistical data
print("\nTotal Statistical Data on Yearly Change in Hydrological Disaster Occurrences: ")
print(hydrological_disaster_change.describe())

# 1980s statistical data
print("\n1980s Statistical Data on Yearly Change in Hydrological Disaster Occurrences: ")
print(hydrological_disaster_change[0:10].describe())

# 1990s statistical data
print("\n1990s Statistical Data on Yearly Change in Hydrological Disaster Occurrences: ")
print(hydrological_disaster_change[10:20].describe())

# 2000s statistical data
print("\n2000s Statistical Data on Yearly Change in Hydrological Disaster Occurrences: ")
print(hydrological_disaster_change[20:30].describe())

# 2010s statistical data
print("\n2010s Statistical Data on Yearly Change in Hydrological Disaster Occurrences: ")
print(hydrological_disaster_change[30:40].describe())

print("\n")

# total statistical changes 1980s-2010s

total_mean_1980s = hydrological_disaster_change[0:10].mean()
total_mean_2010s = hydrological_disaster_change[30:40].mean()
mean_percent_change_1980_to_2020 = 100*(total_mean_2010s-total_mean_1980s)/abs(total_mean_1980s)
print("Percent Change of Mean Hydrological Disaster Occurrences Yearly Change (1980s-2010s): {:.2f}%".format(mean_percent_change_1980_to_2020))

total_max_1980s = hydrological_disaster_change[0:10].max()
total_max_2010s = hydrological_disaster_change[30:40].max()
max_percent_change_1980_to_2020 = 100*(total_max_2010s-total_max_1980s)/abs(total_max_1980s)
print("Percent Change of Min Hydrological Disaster Occurrences Yearly Change (1980s-2010s): {:.2f}%".format(max_percent_change_1980_to_2020))

total_min_1980s = hydrological_disaster_change[0:10].min()
total_min_2010s = hydrological_disaster_change[30:40].min()
min_percent_change_1980_to_2020 = 100*(total_min_2010s-total_min_1980s)/abs(total_min_1980s)
print("Percent Change of Max Hydrological Disaster Occurrences Yearly Change (1980s-2010s): {:.2f}%".format(min_percent_change_1980_to_2020))

total_std_1980s = hydrological_disaster_change[0:10].std()
total_std_2010s = hydrological_disaster_change[30:40].std()
std_percent_change_1980_to_2020 = 100*(total_std_2010s-total_std_1980s)/abs(total_std_1980s)
print("Percent Change of Standard Deviation Hydrological Disaster Occurrences Yearly Change (1980s-2010s): {:.2f}%".format(std_percent_change_1980_to_2020))

In [ ]:
# CHANGE IN FLOOD DISASTER OCCURRENCES ANALYSIS

print("CHANGE IN FLOOD DISASTER OCCURRENCES ANALYSIS")

# total statistical data
print("\nTotal Statistical Data on Yearly Change in Flood Occurrences: ")
print(flood_disaster_change.describe())

# 1980s statistical data
print("\n1980s Statistical Data on Yearly Change in Flood Occurrences: ")
print(flood_disaster_change[0:10].describe())

# 1990s statistical data
print("\n1990s Statistical Data on Yearly Change in Flood Occurrences: ")
print(flood_disaster_change[10:20].describe())

# 2000s statistical data
print("\n2000s Statistical Data on Yearly Change in Flood Occurrences: ")
print(flood_disaster_change[20:30].describe())

# 2010s statistical data
print("\n2010s Statistical Data on Yearly Change in Flood Occurrences: ")
print(flood_disaster_change[30:40].describe())

print("\n")

# total statistical changes 1980s-2010s

total_mean_1980s = flood_disaster_change[0:10].mean()
total_mean_2010s = flood_disaster_change[30:40].mean()
mean_percent_change_1980_to_2020 = 100*(total_mean_2010s-total_mean_1980s)/abs(total_mean_1980s)
print("Percent Change of Mean Flood Occurrences Yearly Change (1980s-2010s): {:.2f}%".format(mean_percent_change_1980_to_2020))

total_max_1980s = flood_disaster_change[0:10].max()
total_max_2010s = flood_disaster_change[30:40].max()
max_percent_change_1980_to_2020 = 100*(total_max_2010s-total_max_1980s)/abs(total_max_1980s)
print("Percent Change of Min Flood Occurrences Yearly Change (1980s-2010s): {:.2f}%".format(max_percent_change_1980_to_2020))

total_min_1980s = flood_disaster_change[0:10].min()
total_min_2010s = flood_disaster_change[30:40].min()
min_percent_change_1980_to_2020 = 100*(total_min_2010s-total_min_1980s)/abs(total_min_1980s)
print("Percent Change of Max Flood Occurrences Yearly Change (1980s-2010s): {:.2f}%".format(min_percent_change_1980_to_2020))

total_std_1980s = flood_disaster_change[0:10].std()
total_std_2010s = flood_disaster_change[30:40].std()
std_percent_change_1980_to_2020 = 100*(total_std_2010s-total_std_1980s)/abs(total_std_1980s)
print("Percent Change of Standard Deviation Flood Occurrences Yearly Change (1980s-2010s): {:.2f}%".format(std_percent_change_1980_to_2020))

In [ ]:
# CHANGE IN LANDSLIDE DISASTER OCCURRENCES ANALYSIS

print("CHANGE IN LANDSLIDE DISASTER OCCURRENCES ANALYSIS")

# total statistical data
print("\nTotal Statistical Data on Yearly Change in Landslide Occurrences: ")
print(landslide_disaster_change.describe())

# 1980s statistical data
print("\n1980s Statistical Data on Yearly Change in Landslide Occurrences: ")
print(landslide_disaster_change[0:10].describe())

# 1990s statistical data
print("\n1990s Statistical Data on Yearly Change in Landslide Occurrences: ")
print(landslide_disaster_change[10:20].describe())

# 2000s statistical data
print("\n2000s Statistical Data on Yearly Change in Landslide Occurrences: ")
print(landslide_disaster_change[20:30].describe())

# 2010s statistical data
print("\n2010s Statistical Data on Yearly Change in Landslide Occurrences: ")
print(landslide_disaster_change[30:40].describe())

print("\n")

# total statistical changes 1980s-2010s

total_mean_1980s = landslide_disaster_change[0:10].mean()
total_mean_2010s = landslide_disaster_change[30:40].mean()
mean_percent_change_1980_to_2020 = 100*(total_mean_2010s-total_mean_1980s)/abs(total_mean_1980s)
print("Percent Change of Mean Landslide Occurrences Yearly Change (1980s-2010s): {:.2f}%".format(mean_percent_change_1980_to_2020))

total_max_1980s = landslide_disaster_change[0:10].max()
total_max_2010s = landslide_disaster_change[30:40].max()
max_percent_change_1980_to_2020 = 100*(total_max_2010s-total_max_1980s)/abs(total_max_1980s)
print("Percent Change of Min Landslide Occurrences Yearly Change (1980s-2010s): {:.2f}%".format(max_percent_change_1980_to_2020))

total_min_1980s = landslide_disaster_change[0:10].min()
total_min_2010s = landslide_disaster_change[30:40].min()
min_percent_change_1980_to_2020 = 100*(total_min_2010s-total_min_1980s)/abs(total_min_1980s)
print("Percent Change of Max Landslide Occurrences Yearly Change (1980s-2010s): {:.2f}%".format(min_percent_change_1980_to_2020))

total_std_1980s = landslide_disaster_change[0:10].std()
total_std_2010s = landslide_disaster_change[30:40].std()
std_percent_change_1980_to_2020 = 100*(total_std_2010s-total_std_1980s)/abs(total_std_1980s)
print("Percent Change of Standard Deviation Landslide Occurrences Yearly Change (1980s-2010s): {:.2f}%".format(std_percent_change_1980_to_2020))

# HYDROLOGICAL DISASTER COSTS ANALYSIS

In [ ]:
flood_costs = flood_data[['Year', 'Total Damages']]
flood_costs = flood_costs.groupby('Year').sum()
flood_costs = flood_costs.squeeze()

landslide_costs = landslide_data[['Year', 'Total Damages']]
landslide_costs = landslide_costs.groupby('Year').sum()
landslide_costs = landslide_costs.squeeze()
landslide_costs = landslide_costs.sort_index()

In [ ]:
# single line plot

plt.grid(axis = 'both')
plt.title("Hydrological Damages per Year\n")
plt.xlabel("\nYear")
plt.ylabel("Hydrological Damages, USD Billion (2020)\n")
plt.plot(hydrological_costs.index, hydrological_costs, linewidth=2, color = 'lightseagreen', alpha = 0.9, label = 'Total Hydrological')
plt.plot(flood_costs.index, flood_costs, linestyle = '--', color = 'darkseagreen', alpha = 0.9, label = 'Flood')
plt.plot(landslide_costs.index, landslide_costs, linestyle = '--', color = 'sienna', alpha = 0.9, label = 'Landslide')

lgd = plt.legend(bbox_to_anchor=(1,1), loc = 'upper left')

plt.savefig("C:/Users/Felix/Documents/GitHub/naturalDisasterAnalysis/Hydrological Disasters/hydrological figures/hydrologicalCostsPerYearSLP.png", bbox_extra_artists=(lgd,), bbox_inches='tight')

In [ ]:
# multiple line plots

fig, axs = plt.subplots(3, sharex = True)
fig.suptitle("Hydrological Damages per Year\n")
axs[1].set_ylabel("Hydrological Damages, USD Billion (2020)\n")
axs[2].set_xlabel("\nYear")

axs[0].plot(hydrological_costs.index, hydrological_costs, linewidth=2, color = 'lightseagreen', alpha = 0.9, label = 'Total Hydrological')
axs[1].plot(flood_costs.index, flood_costs, linestyle = '--', color = 'darkseagreen', alpha = 0.9, label = 'Flood')
axs[2].plot(landslide_costs.index, landslide_costs, linestyle = '--', color = 'sienna', alpha = 0.9, label = 'Landslide')

plt.sca(axs[0])
plt.yticks(np.linspace(0,50,3))
plt.grid(axis = 'both')

plt.sca(axs[1])
plt.yticks(np.linspace(0,50,3))
plt.grid(axis = 'both')

plt.sca(axs[2])
plt.yticks(np.linspace(0,2,3))
plt.grid(axis = 'both')

lgd = fig.legend(bbox_to_anchor=(1,1), loc = 'upper left')

plt.savefig("C:/Users/Felix/Documents/GitHub/naturalDisasterAnalysis/Hydrological Disasters/hydrological figures/hydrologicalCostsPerYearMLP.png", bbox_extra_artists=(lgd,), bbox_inches='tight')

In [ ]:
# single bar chart

plt.grid(axis = 'y')
plt.title("Hydrological Damages per Year\n")
plt.xlabel("\nYear")
plt.ylabel("Hydrological Damages, USD Billion (2020)\n")

plt.bar(flood_costs.index, flood_costs, edgecolor = 'black', color = 'darkseagreen', alpha = 0.9, label = 'Flood')
plt.bar(landslide_costs.index, landslide_costs, bottom = flood_costs, edgecolor = 'black', color = 'sienna', alpha = 0.9, label = 'Landslide')

lgd = plt.legend(bbox_to_anchor=(1,1), loc = 'upper left')

plt.savefig("C:/Users/Felix/Documents/GitHub/naturalDisasterAnalysis/Hydrological Disasters/Hydrological figures/hydrologicalCostsPerYearSBC.png", bbox_extra_artists=(lgd,), bbox_inches='tight')

In [ ]:
# multiple bar charts

fig, axs = plt.subplots(3, sharex = True)
fig.suptitle("Hydrological Damages per Year\n")
axs[1].set_ylabel("Hydrological Damages, USD Billion (2020)\n")
axs[2].set_xlabel("\nYear")

axs[0].bar(hydrological_costs.index, hydrological_costs, edgecolor = 'black', color = 'lightseagreen', alpha = 0.9, label = 'Total Hydrological')
axs[1].bar(flood_costs.index, flood_costs, edgecolor = 'black', color = 'darkseagreen', alpha = 0.9, label = 'Flood')
axs[2].bar(landslide_costs.index, landslide_costs, edgecolor = 'black', color = 'sienna', alpha = 0.9, label = 'Landslide')

plt.sca(axs[0])
plt.yticks(np.linspace(0,50,3))
plt.grid(axis = 'y')

plt.sca(axs[1])
plt.yticks(np.linspace(0,50,3))
plt.grid(axis = 'y')

plt.sca(axs[2])
plt.yticks(np.linspace(0,2,3))
plt.grid(axis = 'y')

lgd = fig.legend(bbox_to_anchor=(1,1), loc = 'upper left')

plt.savefig("C:/Users/Felix/Documents/GitHub/naturalDisasterAnalysis/Hydrological Disasters/Hydrological figures/hydrologicalCostsPerYearMBC.png", bbox_extra_artists=(lgd,), bbox_inches='tight')

# BOXPLOT HYDROLOGICAL DISASTER COSTS

In [ ]:
# BOXPLOT HYDROLOGICAL DISASTER COSTS

hydrological_box_data = [hydrological_costs[0:10], hydrological_costs[10:20], hydrological_costs[20:30], hydrological_costs[30:40]]
box_labels = ['1980s', '1990s', '2000s', '2010s']

plt.figure(figsize=(3,6))
plt.title("Hydrological Damages per Year")
plt.xlabel("Decade")
plt.ylabel("Hydrological Damages, USD Billion (2020)")

plt.boxplot(hydrological_box_data, labels = box_labels, whis = [0, 100])

plt.scatter(x0,hydrological_box_data[0], color = 'lightseagreen', edgecolor = 'black', alpha = 0.9)
plt.scatter(x1,hydrological_box_data[1], color = 'lightseagreen', edgecolor = 'black', alpha = 0.9)
plt.scatter(x2,hydrological_box_data[2], color = 'lightseagreen', edgecolor = 'black', alpha = 0.9)
plt.scatter(x3,hydrological_box_data[3], color = 'lightseagreen', edgecolor = 'black', alpha = 0.9)

plt.savefig("C:/Users/Felix/Documents/GitHub/naturalDisasterAnalysis/Hydrological Disasters/hydrological figures/hydrologicalCostsPerYearBOX.png", bbox_inches='tight')

In [ ]:
# BOXPLOT FLOOD COSTS

flood_box_data = [flood_costs[0:10], flood_costs[10:20], flood_costs[20:30], flood_costs[30:40]]
box_labels = ['1980s', '1990s', '2000s', '2010s']

plt.figure(figsize=(3,6))
plt.title("Flood Damages by Decade")
plt.xlabel("Decade")
plt.ylabel("Flood Damages, USD Billion (2020)")

plt.boxplot(flood_box_data, labels = box_labels, whis = [0, 100])

plt.scatter(x0,flood_box_data[0], color = 'darkseagreen', edgecolor = 'black', alpha = 0.9)
plt.scatter(x1,flood_box_data[1], color = 'darkseagreen', edgecolor = 'black', alpha = 0.9)
plt.scatter(x2,flood_box_data[2], color = 'darkseagreen', edgecolor = 'black', alpha = 0.9)
plt.scatter(x3,flood_box_data[3], color = 'darkseagreen', edgecolor = 'black', alpha = 0.9)

plt.savefig("C:/Users/Felix/Documents/GitHub/naturalDisasterAnalysis/Hydrological Disasters/hydrological figures/floodCostsPerYearBOX.png", bbox_inches='tight')

In [ ]:
# BOXPLOT LANDSLIDE COSTS

landslide_box_data = [landslide_costs[0:10], landslide_costs[10:20], landslide_costs[20:30], landslide_costs[30:40]]
box_labels = ['1980s', '1990s', '2000s', '2010s']

plt.figure(figsize=(3,6))
plt.title("Landslide Damages by Decade")
plt.xlabel("Decade")
plt.ylabel("Landslide Damages, USD Billion (2020)")

plt.boxplot(landslide_box_data, labels = box_labels, whis = [0, 100])

plt.scatter(x0,landslide_box_data[0], color = 'sienna', edgecolor = 'black', alpha = 0.9)
plt.scatter(x1,landslide_box_data[1], color = 'sienna', edgecolor = 'black', alpha = 0.9)
plt.scatter(x2,landslide_box_data[2], color = 'sienna', edgecolor = 'black', alpha = 0.9)
plt.scatter(x3,landslide_box_data[3], color = 'sienna', edgecolor = 'black', alpha = 0.9)

plt.savefig("C:/Users/Felix/Documents/GitHub/naturalDisasterAnalysis/Hydrological Disasters/hydrological figures/landslideCostsPerYearBOX.png", bbox_inches='tight')

# HYDROLOGICAL DISASTER COSTS STATISTICS

In [ ]:
# # HYDROLOGICAL DISASTER COSTS STATISTICS

print("# HYDROLOGICAL DISASTER COSTS STATISTICS")

# hydrological costs statistical data
print("\nTotal Hydrological Costs Statistical Data: ")
print(hydrological_costs.describe())

# 1980s hydrological costs statistical data
print("\n1980s Hydrological Costs Statistical Data: ")
print(hydrological_costs[0:10].describe())

# 1990s hydrological costs statistical data
print("\n1990s Hydrological Costs Statistical Data: ")
print(hydrological_costs[10:20].describe())

# 2000s hydrological costs statistical data
print("\n2000s Hydrological Costs Statistical Data: ")
print(hydrological_costs[20:30].describe())

# 2010s hydrological costs statistical data
print("\n2010s Hydrological Costs Statistical Data: ")
print(hydrological_costs[30:40].describe())

print("\n")

# hydrological costs statistical changes 1980s-2010s
hydrological_mean_costs_1980s = hydrological_costs[0:10].mean()
hydrological_mean_costs_2010s = hydrological_costs[30:40].mean()
mean_costs_percent_change_hydrological_1980_to_2020 = 100*(hydrological_mean_costs_2010s-hydrological_mean_costs_1980s)/hydrological_mean_costs_1980s
print("Mean Hydrological Disaster Costs Change (1980s-2010s): {:.2f}%".format(mean_costs_percent_change_hydrological_1980_to_2020))

hydrological_max_costs_1980s = hydrological_costs[0:10].max()
hydrological_max_costs_2010s = hydrological_costs[30:40].max()
max_costs_percent_change_hydrological_1980_to_2020 = 100*(hydrological_max_costs_2010s-hydrological_max_costs_1980s)/hydrological_max_costs_1980s
print("Max Hydrological Disaster Costs Change (1980s-2010s): {:.2f}%".format(max_costs_percent_change_hydrological_1980_to_2020))

hydrological_min_costs_1980s = hydrological_costs[0:10].min()
hydrological_min_costs_2010s = hydrological_costs[30:40].min()
min_costs_percent_change_hydrological_1980_to_2020 = 100*(hydrological_min_costs_2010s-hydrological_min_costs_1980s)/hydrological_min_costs_1980s
print("Min Hydrological Disaster Costs Change (1980s-2010s): {:.2f}%".format(min_costs_percent_change_hydrological_1980_to_2020))

hydrological_std_costs_1980s = hydrological_costs[0:10].std()
hydrological_std_costs_2010s = hydrological_costs[30:40].std()
std_costs_percent_change_hydrological_1980_to_2020 = 100*(hydrological_std_costs_2010s-hydrological_std_costs_1980s)/hydrological_std_costs_1980s
print("Standard Deviation of Hydrological Disaster Costs Change (1980s-2010s): {:.2f}%\n".format(std_costs_percent_change_hydrological_1980_to_2020))

In [ ]:
# FLOOD COSTS ANALYSIS

print("FLOOD COSTS ANALYSIS")

# flood costs statistical data
print("\nTotal Flood Costs Statistical Data: ")
print(flood_costs.describe())

# 1980s flood costs statistical data
print("\n1980s Flood Costs Statistical Data: ")
print(flood_costs[0:10].describe())

# 1990s flood costs statistical data
print("\n1990s Flood Costs Statistical Data: ")
print(flood_costs[10:20].describe())

# 2000s flood costs statistical data
print("\n2000s Flood Costs Statistical Data: ")
print(flood_costs[20:30].describe())

# 2010s flood costs statistical data
print("\n2010s Flood Costs Statistical Data: ")
print(flood_costs[30:40].describe())

print("\n")

# flood costs statistical changes 1980s-2010s
flood_mean_costs_1980s = flood_costs[0:10].mean()
flood_mean_costs_2010s = flood_costs[30:40].mean()
mean_costs_percent_change_flood_1980_to_2020 = 100*(flood_mean_costs_2010s-flood_mean_costs_1980s)/flood_mean_costs_1980s
print("Mean Flood Disaster Costs Change (1980s-2010s): {:.2f}%".format(mean_costs_percent_change_flood_1980_to_2020))

flood_max_costs_1980s = flood_costs[0:10].max()
flood_max_costs_2010s = flood_costs[30:40].max()
max_costs_percent_change_flood_1980_to_2020 = 100*(flood_max_costs_2010s-flood_max_costs_1980s)/flood_max_costs_1980s
print("Max Flood Disaster Costs Change (1980s-2010s): {:.2f}%".format(max_costs_percent_change_flood_1980_to_2020))

flood_min_costs_1980s = flood_costs[0:10].min()
flood_min_costs_2010s = flood_costs[30:40].min()
min_costs_percent_change_flood_1980_to_2020 = 100*(flood_min_costs_2010s-flood_min_costs_1980s)/flood_min_costs_1980s
print("Min Flood Disaster Costs Change (1980s-2010s): {:.2f}%".format(min_costs_percent_change_flood_1980_to_2020))

flood_std_costs_1980s = flood_costs[0:10].std()
flood_std_costs_2010s = flood_costs[30:40].std()
std_costs_percent_change_flood_1980_to_2020 = 100*(flood_std_costs_2010s-flood_std_costs_1980s)/flood_std_costs_1980s
print("Standard Deviation of Flood Disaster Costs Change (1980s-2010s): {:.2f}%\n".format(std_costs_percent_change_flood_1980_to_2020))

In [ ]:
# LANDSLIDE COSTS ANALYSIS

print("LANDSLIDE COSTS ANALYSIS")

# landslide costs statistical data
print("\nTotal Landslide Costs Statistical Data: ")
print(landslide_costs.describe())

# 1980s landslide costs statistical data
print("\n1980s Landslide Costs Statistical Data: ")
print(landslide_costs[0:10].describe())

# 1990s landslide costs statistical data
print("\n1990s Landslide Costs Statistical Data: ")
print(landslide_costs[10:20].describe())

# 2000s landslide costs statistical data
print("\n2000s Landslide Costs Statistical Data: ")
print(landslide_costs[20:30].describe())

# 2010s landslide costs statistical data
print("\n2010s Landslide Costs Statistical Data: ")
print(landslide_costs[30:40].describe())

print("\n")

# landslide costs statistical changes 1980s-2010s
landslide_mean_costs_1980s = landslide_costs[0:10].mean()
landslide_mean_costs_2010s = landslide_costs[30:40].mean()
mean_costs_percent_change_landslide_1980_to_2020 = 100*(landslide_mean_costs_2010s-landslide_mean_costs_1980s)/landslide_mean_costs_1980s
print("Mean Landslide Disaster Costs Change (1980s-2010s): {:.2f}%".format(mean_costs_percent_change_landslide_1980_to_2020))

landslide_max_costs_1980s = landslide_costs[0:10].max()
landslide_max_costs_2010s = landslide_costs[30:40].max()
max_costs_percent_change_landslide_1980_to_2020 = 100*(landslide_max_costs_2010s-landslide_max_costs_1980s)/landslide_max_costs_1980s
print("Max Landslide Disaster Costs Change (1980s-2010s): {:.2f}%".format(max_costs_percent_change_landslide_1980_to_2020))

landslide_min_costs_1980s = landslide_costs[0:10].min()
landslide_min_costs_2010s = landslide_costs[30:40].min()
min_costs_percent_change_landslide_1980_to_2020 = 100*(landslide_min_costs_2010s-landslide_min_costs_1980s)/landslide_min_costs_1980s
print("Min Landslide Disaster Costs Change (1980s-2010s): {:.2f}%".format(min_costs_percent_change_landslide_1980_to_2020))

landslide_std_costs_1980s = landslide_costs[0:10].std()
landslide_std_costs_2010s = landslide_costs[30:40].std()
std_costs_percent_change_landslide_1980_to_2020 = 100*(landslide_std_costs_2010s-landslide_std_costs_1980s)/landslide_std_costs_1980s
print("Standard Deviation of Landslide Disaster Costs Change (1980s-2010s): {:.2f}%\n".format(std_costs_percent_change_landslide_1980_to_2020))

# YoY CHANGE IN HYDROLOGICAL DISASTER COSTS ANALYSIS

In [ ]:
# CALCULATE YoY CHANGE IN DISASTER COSTS

hydrological_costs_change = [hydrological_costs[i+1]-hydrological_costs[i] for i in range(1980,2020)]
hydrological_costs_change = pd.Series(hydrological_costs_change)

flood_costs_change = [flood_costs[i+1]-flood_costs[i] for i in range(1980,2020)]
flood_costs_change = pd.Series(flood_costs_change)

landslide_costs_change = [landslide_costs[i+1]-landslide_costs[i] for i in range(1980,2020)]
landslide_costs_change = pd.Series(landslide_costs_change)

In [ ]:
# single bar chart

costs_change_data = np.array([flood_costs_change, landslide_costs_change])

data_shape = np.shape(costs_change_data)

# Take negative and positive data apart and cumulate
def get_cumulated_array(costs_change_data, **kwargs):
    cum = costs_change_data.clip(**kwargs)
    cum = np.cumsum(cum, axis=0)
    d = np.zeros(np.shape(costs_change_data))
    d[1:] = cum[:-1]
    return d  

cumulated_data = get_cumulated_array(costs_change_data, min=0)
cumulated_data_neg = get_cumulated_array(costs_change_data, max=0)

# Re-merge negative and positive data.
row_mask = (costs_change_data<0)
cumulated_data[row_mask] = cumulated_data_neg[row_mask]
data_stack = cumulated_data

cols = ["darkseagreen", "sienna"]
labs = ["Flood", "Landslide"]

#fig = plt.figure(figsize=(5,20))
ax = plt.subplot(111)

for i in np.arange(0, data_shape[0]):
    ax.bar(range(1981, 2021), costs_change_data[i], bottom=data_stack[i], edgecolor = 'black', color=cols[i], alpha = 0.9, label = labs[i])

plt.grid(axis = 'y')

plt.title("Year over Year Change in Cost of Hydrological Disasters\n")
plt.xlabel("\nYear")
plt.ylabel("Year over Year Change\n")

lgd = plt.legend(bbox_to_anchor=(1,1), loc = 'upper left')

plt.savefig("C:/Users/Felix/Documents/GitHub/naturalDisasterAnalysis/Hydrological Disasters/hydrological figures/hydrologicalDisasterCostsChangePerYearSBC.png", bbox_extra_artists=(lgd,), bbox_inches='tight')

In [ ]:
# YoY CHANGE IN HYDROLOGICAL DISASTER OCCURRENCES BOXPLOT

total_box_cost_data = [hydrological_costs_change[0:10], hydrological_costs_change[10:20], hydrological_costs_change[20:30], hydrological_costs_change[30:40]]
box_labels = ['1980s', '1990s', '2000s', '2010s']

plt.figure(figsize=(3,6))
plt.title("Year over Year Change in Cost of Hydrological Disasters\n")
plt.xlabel("\nDecade")
plt.ylabel("Year over Year Change\n")

plt.boxplot(total_box_cost_data, labels = box_labels, whis = [0, 100])

plt.scatter(x0,total_box_cost_data[0], color = 'lightseagreen', edgecolor = 'black', alpha = 0.9)
plt.scatter(x1,total_box_cost_data[1], color = 'lightseagreen', edgecolor = 'black', alpha = 0.9)
plt.scatter(x2,total_box_cost_data[2], color = 'lightseagreen', edgecolor = 'black', alpha = 0.9)
plt.scatter(x3,total_box_cost_data[3], color = 'lightseagreen', edgecolor = 'black', alpha = 0.9)

plt.savefig("C:/Users/Felix/Documents/GitHub/naturalDisasterAnalysis/Hydrological Disasters/hydrological figures/hydrologicalCostsChangePerYearBOX.png", bbox_inches='tight')


In [ ]:
# YoY CHANGE IN FLOOD OCCURRENCES BOXPLOT

total_box_cost_data = [flood_costs_change[0:10], flood_costs_change[10:20], flood_costs_change[20:30], flood_costs_change[30:40]]
box_labels = ['1980s', '1990s', '2000s', '2010s']

plt.figure(figsize=(3,6))
plt.title("Year over Year Change in Cost of Floods\n")
plt.xlabel("\nDecade")
plt.ylabel("Year over Year Change\n")

plt.boxplot(total_box_cost_data, labels = box_labels, whis = [0, 100])

plt.scatter(x0,total_box_cost_data[0], color = 'darkseagreen', edgecolor = 'black', alpha = 0.9)
plt.scatter(x1,total_box_cost_data[1], color = 'darkseagreen', edgecolor = 'black', alpha = 0.9)
plt.scatter(x2,total_box_cost_data[2], color = 'darkseagreen', edgecolor = 'black', alpha = 0.9)
plt.scatter(x3,total_box_cost_data[3], color = 'darkseagreen', edgecolor = 'black', alpha = 0.9)

plt.savefig("C:/Users/Felix/Documents/GitHub/naturalDisasterAnalysis/Hydrological Disasters/hydrological figures/floodCostsChangePerYearBOX.png", bbox_inches='tight')


In [ ]:
# YoY CHANGE IN LANDSLIDE OCCURRENCES BOXPLOT

total_box_cost_data = [landslide_costs_change[0:10], landslide_costs_change[10:20], landslide_costs_change[20:30], landslide_costs_change[30:40]]
box_labels = ['1980s', '1990s', '2000s', '2010s']

plt.figure(figsize=(3,6))
plt.title("Year over Year Change in Cost of floods\n")
plt.xlabel("\nDecade")
plt.ylabel("Year over Year Change\n")

plt.boxplot(total_box_cost_data, labels = box_labels, whis = [0, 100])

plt.scatter(x0,total_box_cost_data[0], color = 'sienna', edgecolor = 'black', alpha = 0.9)
plt.scatter(x1,total_box_cost_data[1], color = 'sienna', edgecolor = 'black', alpha = 0.9)
plt.scatter(x2,total_box_cost_data[2], color = 'sienna', edgecolor = 'black', alpha = 0.9)
plt.scatter(x3,total_box_cost_data[3], color = 'sienna', edgecolor = 'black', alpha = 0.9)

plt.savefig("C:/Users/Felix/Documents/GitHub/naturalDisasterAnalysis/hydrological Disasters/hydrological figures/landslideCostsChangePerYearBOX.png", bbox_inches='tight')


# YoY CHANGE IN HYDROLOGICAL DISASTER COSTS STATISTICS

In [ ]:
# CHANGE IN HYDROLOGICAL DISASTER COSTS ANALYSIS

print("CHANGE IN HYDROLOGICAL DISASTER COSTS ANALYSIS")

# total statistical data
print("\nTotal Statistical Data on Yearly Change in Hydrological Disaster Costs: ")
print(hydrological_costs_change.describe())

# 1980s statistical data
print("\n1980s Statistical Data on Yearly Change in Hydrological Disaster Costs: ")
print(hydrological_costs_change[0:10].describe())

# 1990s statistical data
print("\n1990s Statistical Data on Yearly Change in Hydrological Disaster Costs: ")
print(hydrological_costs_change[10:20].describe())

# 2000s statistical data
print("\n2000s Statistical Data on Yearly Change in Hydrological Disaster Costs: ")
print(hydrological_costs_change[20:30].describe())

# 2010s statistical data
print("\n2010s Statistical Data on Yearly Change in Hydrological Disaster Costs: ")
print(hydrological_costs_change[30:40].describe())

print("\n")

# total statistical changes 1980s-2010s

total_mean_1980s = hydrological_costs_change[0:10].mean()
total_mean_2010s = hydrological_costs_change[30:40].mean()
mean_percent_change_1980_to_2020 = 100*(total_mean_2010s-total_mean_1980s)/abs(total_mean_1980s)
print("Percent Change of Mean Hydrological Disaster Costs Yearly Change (1980s-2010s): {:.2f}%".format(mean_percent_change_1980_to_2020))

total_max_1980s = hydrological_costs_change[0:10].max()
total_max_2010s = hydrological_costs_change[30:40].max()
max_percent_change_1980_to_2020 = 100*(total_max_2010s-total_max_1980s)/abs(total_max_1980s)
print("Percent Change of Min Hydrological Disaster Costs Yearly Change (1980s-2010s): {:.2f}%".format(max_percent_change_1980_to_2020))

total_min_1980s = hydrological_costs_change[0:10].min()
total_min_2010s = hydrological_costs_change[30:40].min()
min_percent_change_1980_to_2020 = 100*(total_min_2010s-total_min_1980s)/abs(total_min_1980s)
print("Percent Change of Max Hydrological Disaster Costs Yearly Change (1980s-2010s): {:.2f}%".format(min_percent_change_1980_to_2020))

total_std_1980s = hydrological_costs_change[0:10].std()
total_std_2010s = hydrological_costs_change[30:40].std()
std_percent_change_1980_to_2020 = 100*(total_std_2010s-total_std_1980s)/abs(total_std_1980s)
print("Percent Change of Standard Deviation Hydrological Disaster Costs Yearly Change (1980s-2010s): {:.2f}%".format(std_percent_change_1980_to_2020))

In [ ]:
# CHANGE IN FLOOD DISASTER COSTS ANALYSIS

print("CHANGE IN FLOOD DISASTER COSTS ANALYSIS")

# total statistical data
print("\nTotal Statistical Data on Yearly Change in Flood Disaster Costs: ")
print(flood_costs_change.describe())

# 1980s statistical data
print("\n1980s Statistical Data on Yearly Change in Flood Disaster Costs: ")
print(flood_costs_change[0:10].describe())

# 1990s statistical data
print("\n1990s Statistical Data on Yearly Change in Flood Disaster Costs: ")
print(flood_costs_change[10:20].describe())

# 2000s statistical data
print("\n2000s Statistical Data on Yearly Change in Flood Disaster Costs: ")
print(flood_costs_change[20:30].describe())

# 2010s statistical data
print("\n2010s Statistical Data on Yearly Change in Flood Disaster Costs: ")
print(flood_costs_change[30:40].describe())

print("\n")

# total statistical changes 1980s-2010s

total_mean_1980s = flood_costs_change[0:10].mean()
total_mean_2010s = flood_costs_change[30:40].mean()
mean_percent_change_1980_to_2020 = 100*(total_mean_2010s-total_mean_1980s)/abs(total_mean_1980s)
print("Percent Change of Mean Flood Disaster Costs Yearly Change (1980s-2010s): {:.2f}%".format(mean_percent_change_1980_to_2020))

total_max_1980s = flood_costs_change[0:10].max()
total_max_2010s = flood_costs_change[30:40].max()
max_percent_change_1980_to_2020 = 100*(total_max_2010s-total_max_1980s)/abs(total_max_1980s)
print("Percent Change of Min Flood Disaster Costs Yearly Change (1980s-2010s): {:.2f}%".format(max_percent_change_1980_to_2020))

total_min_1980s = flood_costs_change[0:10].min()
total_min_2010s = flood_costs_change[30:40].min()
min_percent_change_1980_to_2020 = 100*(total_min_2010s-total_min_1980s)/abs(total_min_1980s)
print("Percent Change of Max Flood Disaster Costs Yearly Change (1980s-2010s): {:.2f}%".format(min_percent_change_1980_to_2020))

total_std_1980s = flood_costs_change[0:10].std()
total_std_2010s = flood_costs_change[30:40].std()
std_percent_change_1980_to_2020 = 100*(total_std_2010s-total_std_1980s)/abs(total_std_1980s)
print("Percent Change of Standard Deviation Flood Disaster Costs Yearly Change (1980s-2010s): {:.2f}%".format(std_percent_change_1980_to_2020))

In [ ]:
# CHANGE IN LANDSLIDE COSTS ANALYSIS

print("CHANGE IN LANDSLIDE COSTS ANALYSIS")

# total statistical data
print("\nTotal Statistical Data on Yearly Change in Landslide Costs: ")
print(landslide_costs_change.describe())

# 1980s statistical data
print("\n1980s Statistical Data on Yearly Change in Landslide Costs: ")
print(landslide_costs_change[0:10].describe())

# 1990s statistical data
print("\n1990s Statistical Data on Yearly Change in Landslide Costs: ")
print(landslide_costs_change[10:20].describe())

# 2000s statistical data
print("\n2000s Statistical Data on Yearly Change in Landslide Costs: ")
print(landslide_costs_change[20:30].describe())

# 2010s statistical data
print("\n2010s Statistical Data on Yearly Change in Landslide Costs: ")
print(landslide_costs_change[30:40].describe())

print("\n")

# total statistical changes 1980s-2010s

total_mean_1980s = landslide_costs_change[0:10].mean()
total_mean_2010s = landslide_costs_change[30:40].mean()
mean_percent_change_1980_to_2020 = 100*(total_mean_2010s-total_mean_1980s)/abs(total_mean_1980s)
print("Percent Change of Mean Landslide Costs Yearly Change (1980s-2010s): {:.2f}%".format(mean_percent_change_1980_to_2020))

total_max_1980s = landslide_costs_change[0:10].max()
total_max_2010s = landslide_costs_change[30:40].max()
max_percent_change_1980_to_2020 = 100*(total_max_2010s-total_max_1980s)/abs(total_max_1980s)
print("Percent Change of Min Landslide Costs Yearly Change (1980s-2010s): {:.2f}%".format(max_percent_change_1980_to_2020))

total_min_1980s = landslide_costs_change[0:10].min()
total_min_2010s = landslide_costs_change[30:40].min()
min_percent_change_1980_to_2020 = 100*(total_min_2010s-total_min_1980s)/abs(total_min_1980s)
print("Percent Change of Max Landslide Costs Yearly Change (1980s-2010s): {:.2f}%".format(min_percent_change_1980_to_2020))

total_std_1980s = landslide_costs_change[0:10].std()
total_std_2010s = landslide_costs_change[30:40].std()
std_percent_change_1980_to_2020 = 100*(total_std_2010s-total_std_1980s)/abs(total_std_1980s)
print("Percent Change of Standard Deviation Landslide Costs Yearly Change (1980s-2010s): {:.2f}%".format(std_percent_change_1980_to_2020))